In [9]:
import numpy as np
import tensorflow as tf
from PIL import Image

In [10]:
background = np.zeros((1000,1000),dtype=bool)

In [11]:
# matrix into image file
def bg2img(bg):
    img = np.zeros((bg.shape[0],bg.shape[1],3),dtype=np.uint8)
    img[bg==True]=(0,0,0)
    img[bg==False]=(255,255,255)
    img = Image.fromarray(img, 'RGB')
    img.save('background.png')
    img.show()
    
    

In [12]:
machine1 = np.ones((100,300),dtype=bool)
machine1[0:50,:100]=False
move=[100,200]
background[move[0]:move[0]+machine1.shape[0],move[1]:move[1]+machine1.shape[1]] = machine1
bg2img(background)

In [13]:
def collide_check(bg,machine):
    if 2 in bg+machine:
        return True
    return False
    

In [14]:
def machine_batch(bg,machine):
    bg = bg+machine
    if collide_check(bg):
        raise Exception('collide')
    return bg

In [16]:
background.shape[0]

1000

In [21]:
2 in np.zeros([1])

False

In [24]:
np.array([2])

array([2])

In [26]:
M = np.array([[]])

In [27]:
M[0,0] = 2

IndexError: index 0 is out of bounds for axis 1 with size 0